<img src="files/casa.jpg">

WORKSHOP_02: riproduzione di una struttura rinforzata
http://www.secrets-of-shed-building.com/images/xshed-roof-construction1.jpg.pagespeed.ic.j_pWZsEVfw.jpg

Importo le librerie necessarie

In [45]:
from pyplasm import *
from numpy import *
from math import *

Riscrivo la funzione relativa al workshop_01 (telaio "piano") e la testo

In [46]:
def flatStructure(xBeam,zBeam,xPillar,yPillar,beamPriority,distancesPillars,distancesBeams):
	
	dp = [yPillar]
	for val in distancesPillars:
		dp.append(-val+yPillar)
		dp.append(yPillar)

	db = []
	for val in distancesBeams:
		db.append(-val+zBeam)
		db.append(zBeam)

	dpn = []
	dbn = []

	dpsum = 0
	for val in dp:
    		dpsum += abs(val)
		dpn.append(-val)

	dbsum = 0
	for val in db:
    		dbsum += abs(val)
		dbn.append(-val)

	arrayA = []
	arrayB = []

	if beamPriority:
		arrayA = dbn
		arrayB.append(dpsum) 
	else:
		arrayA.append(dbsum)
		arrayB= dpn

	y = QUOTE(dp)
	ya = QUOTE(arrayA)
	a = PROD([y,ya])
	
	zb = QUOTE(db)
	z= QUOTE(arrayB)
	b = PROD([z,zb])

	xa = QUOTE([xPillar])

	xb = QUOTE([xBeam])

	return STRUCT([T(1)(-xPillar/2),PROD([xa,a]),T(1)(-(xBeam-xPillar)/2),PROD([xb,b])])

VIEW(flatStructure(0.8,0.2,0.3,0.3,1,[1,2,3],[1,2,3]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000764CAB0> >

Definisco una funzione che prende come parametro il nome di un file ed assembla una struttura (e la testo)

In [47]:
def fileToStructure(file_name):
    import csv

    dimensions = []
    distancesPillars = []
    distancesBeams = []
    toFill = dimensions
    vector = [0,0,0]
    structure = STRUCT([CUBE(0)])

    i = 1

    with open(file_name, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        
        for row in spamreader:
                if(i % 2 == 0):
                    j = 0
                    for val in row:
                        val = float(val)
                        if(j == 5): toFill=distancesPillars
                        if(val == -1): toFill=distancesBeams
                        else: toFill.append(val)
                        j+=1
                    newFlatStructure = flatStructure(dimensions[0],dimensions[1],dimensions[2],dimensions[3],dimensions[4],distancesPillars,distancesBeams)
                    structure = STRUCT([structure,T([1,2,3])(vector),newFlatStructure])
                    dimensions = []
                    distancesPillars = []
                    distancesBeams = []
                    toFill = dimensions
                else:
                    z = 0
                    for val in row: 
                        vector[z] += float(val)
                        z +=1
            
                i+=1
    return structure

VIEW(fileToStructure('frame_data_446724.csv'))
    

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000765D5D0> >

Creo una struttura utile a riprodurre quella del modello scelto

In [48]:
myStructure = fileToStructure('frame_data_446724.csv')

Aggiungo le travi su asse X e visualizzo

In [49]:
xPillar = 0.3
xBeam = 0.8
distancePillarsAxis = 3
distancePillars = distancePillarsAxis-xPillar 
vectorDistancesX = [distancePillars,-xPillar,distancePillars,-xPillar,distancePillars,-xPillar,distancePillars]
vecX = QUOTE(vectorDistancesX)
vecXY = PROD ([vecX,QUOTE([.4])])
vecXYZ = COLOR(RED)(PROD([vecXY,QUOTE([.1])]))
axesOnX = STRUCT([T([1])([2.15]),vecXYZ])

myStructure = STRUCT([myStructure,T([2,3])([0,0]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([8,0]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([0,3]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([2,3]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([4,3]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([6,3]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([2,6]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([4,6]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([6,6]),axesOnX])
myStructure = STRUCT([myStructure,T([2,3])([8,3]),axesOnX])

VIEW(myStructure)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000764CCF0> >

Aggiungo il basamento e visualizzo

In [50]:
myStructure = STRUCT([CUBOID([15,8+0.4,0.5]),T(3)(0.5),myStructure])
VIEW(myStructure)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000764CD50> >

Aggiungo il tetto e visualizzo (RISULTATO FINALE)

In [51]:
from math import *
pi = math.pi

zRoof = 0.2
yRoof = 8.4
xBoardRoof = xBeam

ht = CUBOID([xBoardRoof,yRoof/sqrt(3),zRoof])
t1 = R([2,3])(pi/6)(ht)
t2 = R([2,3])(pi/6*5)(ht)
boardRoof = STRUCT([t1,T([2,3])([yRoof+zRoof/2,zRoof*sqrt(3)/2]),t2])

roof = STRUCT([T(1)(1.6),boardRoof,T(1)(3),boardRoof,T(1)(3),boardRoof,T(1)(3),boardRoof,T(1)(3),boardRoof])


myStructure = STRUCT([myStructure,T(3)(6.5),roof])

VIEW(myStructure)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000763DB40> >

Definisco funzione richiesta e la testo

In [52]:
def ggpl_bone_structure (file_name):
    myStructure = myStructure = fileToStructure(file_name)
    
    xPillar = 0.3
    xBeam = 0.8
    distancePillarsAxis = 3
    distancePillars = distancePillarsAxis-xPillar 
    vectorDistancesX = [distancePillars,-xPillar,distancePillars,-xPillar,distancePillars,-xPillar,distancePillars]
    vecX = QUOTE(vectorDistancesX)
    vecXY = PROD ([vecX,QUOTE([.4])])
    vecXYZ = COLOR(RED)(PROD([vecXY,QUOTE([.1])]))
    axesOnX = STRUCT([T([1])([2.15]),vecXYZ])

    myStructure = STRUCT([myStructure,T([2,3])([0,0]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([8,0]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([0,3]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([2,3]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([4,3]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([6,3]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([2,6]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([4,6]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([6,6]),axesOnX])
    myStructure = STRUCT([myStructure,T([2,3])([8,3]),axesOnX])
    
    myStructure = STRUCT([CUBOID([15,8+0.4,0.5]),T(3)(0.5),myStructure])
    
    pi = math.pi

    zRoof = 0.2
    yRoof = 8.4
    xBoardRoof = xBeam

    ht = CUBOID([xBoardRoof,yRoof/sqrt(3),zRoof])
    t1 = R([2,3])(pi/6)(ht)
    t2 = R([2,3])(pi/6*5)(ht)
    boardRoof = STRUCT([t1,T([2,3])([yRoof+zRoof/2,zRoof*sqrt(3)/2]),t2])

    roof = STRUCT([T(1)(1.6),boardRoof,T(1)(3),boardRoof,T(1)(3),boardRoof,T(1)(3),boardRoof,T(1)(3),boardRoof])


    myStructure = STRUCT([myStructure,T(3)(6.5),roof])
    
    return myStructure
    
VIEW(ggpl_bone_structure('frame_data_446724.csv'))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000076C4810> >